In [ ]:
import numpy
import pandas as pd

from configuration import Config, FilePaths

FILE_PATHS = FilePaths()

In [ ]:
rearranged_df_as_read = pd.read_csv(FILE_PATHS.REARRANGED_DATA, dtype=str)

In [ ]:
rearranged_df = rearranged_df_as_read.copy()
rearranged_df.set_index("Patient Id", inplace=True)
rearranged_df.head()

In [ ]:
def fit_data_to_timesteps(
    df, verbose: bool = False, pad_timesteps: bool = True
) -> list[list]:
    patient_id_set: set = set(df.index)
    time_stamps: list = [str(x) for x in range(1, 365 + 1)]
    ehr_stream: list = []
    for patient_id in patient_id_set:
        element = df.index.get_loc(patient_id)
        if verbose is True:
            print("-" * 56)
            print(f"Patient: {element}")
            print("-" * 56)
        for (colname, colvalue) in df.iloc[element].iteritems():
            if colvalue not in time_stamps:
                end_timestep: int = int(Config.time_step) + int(colname)
                clamped_end_timestep: int = numpy.clip(end_timestep, 0, 365)
                row_slice = df.iloc[element]
                if verbose is True:
                    print(f"Visit found")
                    print(f"Timestep with entry: {colname}")
                    print(f"Timestep to go up till: { clamped_end_timestep}")
                    print("_" * 56)
                data_stream = row_slice.iloc[
                    int(colname) : int(clamped_end_timestep)
                ].to_list()

                ehr_stream.append(
                    row_slice.iloc[int(colname) : int(clamped_end_timestep)].to_list()
                )
    return ehr_stream


ehr_stream: list = fit_data_to_timesteps(rearranged_df)

# To do - add in pad_timestep function, and add counting from previous filled timesteps.

In [ ]:
ehr_stream

lengths = [len(i) for i in ehr_stream]

In [ ]:
lengths